graph db
conjuctive regaulr path queires poptimization


https://arxiv.org/pdf/2407.04823 Path-based Algebraic Foundations of Graph Query Languages
https://openproceedings.org/2025/conf/edbt/paper-234.pdf Path-based Algebraic Foundations of Graph Query Languages

https://dl.acm.org/doi/abs/10.1145/3725237 Minimizing Conjunctive Regular Path Queries